Выполнил студент группы P33211 Никитин В.В.

Мини-игра бокс

Суть написанной игры в использовании датчиков для считывания ЭМГ сигналов с игрока и в соответствии с его движениями определять - победил ли он в раунде соперника или проиграл.

In [9]:
!pip install neurokit2

тестовые данные ЭМГ сигналов


In [20]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1ZZatZ2SJqz6ProhwS90rz001yKUK3l2d' -O archive.zip
!unzip archive.zip -d /content

--2024-05-29 07:46:04--  https://drive.google.com/uc?export=download&id=1ZZatZ2SJqz6ProhwS90rz001yKUK3l2d
Resolving drive.google.com (drive.google.com)... 142.251.183.102, 142.251.183.113, 142.251.183.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.183.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ZZatZ2SJqz6ProhwS90rz001yKUK3l2d&export=download [following]
--2024-05-29 07:46:04--  https://drive.usercontent.google.com/download?id=1ZZatZ2SJqz6ProhwS90rz001yKUK3l2d&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.171.132, 2607:f8b0:4001:c5f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.171.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348861 (341K) [application/octet-stream]
Saving to: ‘archive.zip’

archive.zip         100%[===================>] 340.68K  --.-KB

In [10]:
import random
import pandas as pd
import neurokit2 as nk
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

Механика игры на данном этапе без виртуального соперника в том, что выпадает случайная комбинация из двух составляющих, которые игрок использует при взаимодействии с виртуальной системой - датчики и вр очки, например. Также есть режим демо для автоматической игры пользователя для демонстрации

In [11]:

# Определение движений и связанных с ними мышц
moves = {
    "сильный блок": [("плечелучевая", "грудь")],
    "слабый блок": [("плечелучевая", "")],
    "джеб": [("плечо", "")],
    "сильный джеб": [("плечо", "трицепс")],
    "хук": [("грудь", "плечо")],
    "апперкот": [("грудь", "бицепс")]
}

# Возможные комбинации мышц
muscles = ["плечелучевая", "грудь", "плечо", "трицепс", "бицепс", ""]

# Базовые пороговые значения для амплитуды
thresholds = {
    "плечелучевая": 0.55,
    "грудь": 0.5,
    "плечо": 0.6,
    "трицепс": 0.5,
    "бицепс": 0.6,
    "": 0  # Для случаев, когда одна из мышц не указана
}



In [12]:
def set_difficulty(difficulty):
    modifier = 0
    if difficulty == "легкий":
        modifier = -0.15
    elif difficulty == "тяжелый":
        modifier = 0.15
    elif difficulty == "демо":
        modifier = -0.25
    return {muscle: max(0, threshold + modifier) for muscle, threshold in thresholds.items()}



In [13]:
def generate_random_muscle_combination():
    muscle1 = random.choice(muscles)
    muscle2 = random.choice(muscles)
    return (muscle1, muscle2)

def is_valid_combination(muscle_combination):
    if muscle_combination == ("", ""):
        return False

    for move, combinations in moves.items():
        if muscle_combination in combinations or muscle_combination[::-1] in combinations:
            return True
    return False

def check_move_success(muscle_combination, amplitude, muscle_thresholds):
    for move, combinations in moves.items():
        if muscle_combination in combinations or muscle_combination[::-1] in combinations:
            threshold = max(muscle_thresholds.get(muscle_combination[0], 0), muscle_thresholds.get(muscle_combination[1], 0))
            return amplitude > threshold, move
    return False, "пропуск"

def play_round(muscle_combination, amplitudes, muscle_thresholds):
    amplitude = max(amplitudes) if len(amplitudes) > 1 else amplitudes[0]
    success, move = check_move_success(muscle_combination, amplitude, muscle_thresholds)

    if success:
        result = f"Успешно выполнено движение: {move} с амплитудой {amplitude:.2f}"
    else:
        result = f"Неудача при выполнении движения: {move} с амплитудой {amplitude:.2f}"
    return result


def game_mode_file(muscle_thresholds):
    rounds = int(input("Введите количество раундов: "))
    for i in range(rounds):
        print(f"\nРаунд {i + 1}")
        muscle_combination = generate_random_muscle_combination()
        print(f"Выпала комбинация мышц: {muscle_combination}")

        if not is_valid_combination(muscle_combination):
            print(f"Неудача: комбинация {muscle_combination} не подходит")
            continue

        amplitudes = []
        for muscle in muscle_combination:
            if muscle:
                data_path = input(f"Введите путь к вашему CSV файлу с амплитудой для {muscle}: ")
                amplitude = load_amplitude_from_csv(data_path)
                amplitudes.append(amplitude)

        print(play_round(muscle_combination, amplitudes, muscle_thresholds))

def game_mode_auto(muscle_thresholds):
    rounds = int(input("Введите количество раундов: "))
    for i in range(rounds):
        print(f"\nРаунд {i + 1}")
        muscle_combination = generate_random_muscle_combination()
        print(f"Выпала комбинация мышц: {muscle_combination}")

        if not is_valid_combination(muscle_combination):
            print(f"Неудача: комбинация {muscle_combination} не подходит")
            continue

        amplitudes = [generate_amplitude_with_neurokit2() for _ in muscle_combination if muscle_combination]
        print(play_round(muscle_combination, amplitudes, muscle_thresholds))



In [14]:
# Алгоритм лабораторной работы для поиска амплитуды
def load_amplitude_from_csv(data_path):
    def butterworth_filter(y, low, high=None, btype='bandpass', order=4, fs=300):
        nyquist = 0.5 * fs
        low = low / nyquist
        high = high / nyquist
        b, a = signal.butter(order, [low, high], btype=btype)
        return signal.lfilter(b, a, y)

    def rectify(y):
        return np.abs(y)

    def butterworth_3rd_filter(y, f=0.02):
        b, a = signal.butter(3, f)
        return signal.lfilter(b, a, y)

    # Загрузка данных из CSV
    data = pd.read_csv(data_path, delimiter=';')
    t = data.iloc[:, 0].values
    y = data.iloc[:, 1].values

    # Маскирование данных
    y = y[y <= 4]

    # Фильтрация данных
    y = butterworth_filter(y, 40, 60, 'stop')
    y = butterworth_filter(y, 5, 100, 'bandpass')

    # Ректификация
    y = rectify(y)

    # Сглаживание и выбор пиков
    smoothed_y = butterworth_3rd_filter(y, 0.01)
    amplitude = smoothed_y.max()

    return amplitude

def generate_amplitude_with_neurokit2():
    emg_signal = nk.emg_simulate(duration=10, sampling_rate=1000, burst_number=5)
    emg_amplitude = nk.emg_amplitude(emg_signal)
    return emg_amplitude.max()

In [18]:
def main():
    difficulty = input("Выберите уровень сложности (легкий, средний, тяжелый, демо): ")
    muscle_thresholds = set_difficulty(difficulty)

    if difficulty == "демо":
        game_mode_auto(muscle_thresholds)
    else:
        mode = input("Выберите режим игры (1 - загрузка файлов, 2 - авто генерация сигналов): ")
        if mode == '1':
            game_mode_file(muscle_thresholds)
        elif mode == '2':
            game_mode_auto(muscle_thresholds)
        else:
            print("Неверный режим. Пожалуйста, выберите 1 или 2.")

if __name__ == "__main__":
    main()

Выберите уровень сложности (легкий, средний, тяжелый, демо): демо
Введите количество раундов: 30

Раунд 1
Выпала комбинация мышц: ('плечо', 'плечелучевая')
Неудача: комбинация ('плечо', 'плечелучевая') не подходит

Раунд 2
Выпала комбинация мышц: ('плечо', 'плечо')
Неудача: комбинация ('плечо', 'плечо') не подходит

Раунд 3
Выпала комбинация мышц: ('плечелучевая', 'трицепс')
Неудача: комбинация ('плечелучевая', 'трицепс') не подходит

Раунд 4
Выпала комбинация мышц: ('плечелучевая', 'плечелучевая')
Неудача: комбинация ('плечелучевая', 'плечелучевая') не подходит

Раунд 5
Выпала комбинация мышц: ('грудь', 'плечо')
Успешно выполнено движение: хук с амплитудой 0.40

Раунд 6
Выпала комбинация мышц: ('плечо', '')
Успешно выполнено движение: джеб с амплитудой 0.38

Раунд 7
Выпала комбинация мышц: ('грудь', 'грудь')
Неудача: комбинация ('грудь', 'грудь') не подходит

Раунд 8
Выпала комбинация мышц: ('', 'бицепс')
Неудача: комбинация ('', 'бицепс') не подходит

Раунд 9
Выпала комбинация мышц: 

In [19]:
if __name__ == "__main__":
    main()

Выберите уровень сложности (легкий, средний, тяжелый, демо): тяжелый
Выберите режим игры (1 - загрузка файлов, 2 - авто генерация сигналов): 1
Введите количество раундов: 7

Раунд 1
Выпала комбинация мышц: ('', 'плечелучевая')
Введите путь к вашему CSV файлу с амплитудой для плечелучевая: /content/input1.csv
Успешно выполнено движение: слабый блок с амплитудой 1.02

Раунд 2
Выпала комбинация мышц: ('плечо', 'грудь')
Введите путь к вашему CSV файлу с амплитудой для плечо: /content/input15.csv
Введите путь к вашему CSV файлу с амплитудой для грудь: /content/input2.csv
Успешно выполнено движение: хук с амплитудой 0.86

Раунд 3
Выпала комбинация мышц: ('грудь', 'грудь')
Неудача: комбинация ('грудь', 'грудь') не подходит

Раунд 4
Выпала комбинация мышц: ('', 'грудь')
Неудача: комбинация ('', 'грудь') не подходит

Раунд 5
Выпала комбинация мышц: ('грудь', 'плечо')
Введите путь к вашему CSV файлу с амплитудой для грудь: /content/input2.csv
Введите путь к вашему CSV файлу с амплитудой для плеч